<a href="https://colab.research.google.com/github/ozgung/microgpt-Turkce/blob/main/microgpt_T%C3%BCrk%C3%A7e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> GPT’yi saf, bağımlılıksız Python ile eğitmenin ve çıkarım yapmanın en “atomik” yolu.\
> Bu dosya algoritmanın tamamı.\
> Geri kalan her şey sadece verimlilik.\
> @karpathy

200 satırda GPT eğiten ve çıkarım (inference) yapan microgpt projesinin Türkçe açıklamasına hoşgeldiniz. Aşağıda kodu aşama aşama inceleyeceğiz.

İngilizce orijinal [Colab](https://colab.research.google.com/drive/1vyN5zo6rqUp_dYNbT4Yrco66zuWCZKoN?usp=sharing) ve
[Blog yazısına](https://karpathy.github.io/2026/02/12/microgpt/) linklerden ulaşabilirsiniz.

Bu proje en yalın haliyle Python kullanıyor ve dışarıdan hiçbir hazır kütüphane kullanmayacağız. Standart ktüpohaneden de yalnızca logaritma (`math.log`) ve üstel fonksiyonlar (`math.exp`) ile rastgele sayı modülünü (`random`) import ederek başlıyoruz. `os.path.exists` ve `urllib.request.urlretrieve` fonksiyonlarını ise sadece örnek veri dosyasını indirmek için kullanıyoruz.

`random.seed(42)` rastgele fonksiyonların her zaman aynı sayıları aynı sırada üretmesini sağlayarak bize yardımcı olacak. Böylece hep aynı sonuçları göreceğiz. Bu sadece geliştirme sırasında hataları kolay bulabilmemiz için gerekli. Normalde bu satırı kaldırabiliriz.


In [3]:
'''Translated to Turkish by ChatGPT from the orginal in English.'''
'''İngilizce orijinalinden ChatGPT ile Türkçe'ye çevrilmiştir.'''
'''https://colab.research.google.com/drive/1vyN5zo6rqUp_dYNbT4Yrco66zuWCZKoN?usp=sharing'''
'''https://karpathy.github.io/2026/02/12/microgpt/'''

import os       # os.path.exists
import math     # math.log, math.exp
import random   # random.seed, random.choices, random.gauss, random.shuffle
random.seed(42) # Kaos içinde düzen olsun

## Eğitim Verisi
Elimizde örnek olarak basit bir veriseti var. Linke tıklayarak içeriğini görebilirsiniz. Her satırda bir insan ismi olan bir liste bu. Aşağıda önce bu listeyi indirip `input.txt` dosyasına kaydediyoruz. Sonra dosyayı okuyup `docs` isimli listeyi oluşturuyoruz. 32033 elemanlı bu isim listesi eğitim için bizim veri setimiz olacak.

Gerçek GPT'de bu `docs` listesinin her elemanı bir dökümanı, örneğin bir web sayfasındaki metinleri içerecek. Bu örnekte ise basit bir örnek olarak insan isimleri kullanılmış.

In [5]:

# Bir giriş veri kümesi `docs` olsun: dokümanlardan oluşan liste[str] (ör. isimlerden oluşan bir veri kümesi)
if not os.path.exists('input.txt'):
    import urllib.request
    names_url = 'https://raw.githubusercontent.com/karpathy/makemore/refs/heads/master/names.txt'
    urllib.request.urlretrieve(names_url, 'input.txt')
docs = [l.strip() for l in open('input.txt').read().strip().split('\n') if l.strip()] # dokümanlardan oluşan liste[str]
random.shuffle(docs)
print(f"doküman sayısı: {len(docs)}")


doküman sayısı: 32033


# Tokenleştirme
Bu liste sözcüklerden yani harflerden oluşuyor. Fakat yapay öğrenme modeli için bize sayılar gerekiyor. Girdileri nasıl sayılara dönüştürebiliriz?
Basit bir yöntem her harfe bir sayı atamak. İngiliz alfabesinde 26 harf var. Bir tane de dizinin başlangıcını tanımlayan özel bir sembol (token) tanımlıyoruz. Bu özel sembolünün ismi orijınal kodda BOS (Beginning of Sequence, Dizinin Başlangıcı) olarak isimlendirilmiş. Ben bunu başlangıç anlamında BAS olarak değiştirdim ki daha anlaşılır olsun. Model bu BAS tokenini ayraç olarak kullanmayı öğrenecek. Farkli isimleri (dökümanları) bu şekilde birbirinden ayıracağız. Örnek:
`
[BAS, e, m, m, a, BAS]
`

Burada **token** kavramına da değinelim. Dokumanı ayırdıgımız bu parçacıklara genelde token adı veriliyor. Biz basitçe her harf için bir token gibi bir eşleştirme yaptık. Her tokeni de bir tamsayı ile temsil ettik (token id). Gerçek GPT'de daha etkili bir yol olarak harf gruplarını tokena çeviriyorlar. GPT4'ün kullandığı token dönüştrücüsü [tiktoken](https://github.com/openai/tiktoken)'a buradan ulaşabilirsiniz.

In [6]:
# Dizgileri ayrık sembollere çeviren ve geri dönüştüren bir Tokenizer olsun
uchars = sorted(set(''.join(docs))) # veri kümesindeki benzersiz karakterler token id’leri 0..n-1 olur
BAS = len(uchars) # özel “Dizinin Başlangıcı” (BAS) token’ı için token id
vocab_size = len(uchars) + 1 # toplam benzersiz token sayısı, +1 BAS içindir
print(f"sözlük boyutu: {vocab_size}")

sözlük boyutu: 27


## Otomatik Türev (AutoGrad)
Geldik matematiksel olarak en yoğun ve belki de asıl işi yapan kısma. Normalde `pytorch`, `JAX` gibi kütüphanelerin en büyük katkısı da bu.

Yapay Sinirsel Ağlar günümüzde Geri Yayılım (Backpropagation) algoritması ile eğitiliyor. Bütün ağı bir hesaplama grafiği olarak tanımlıyoruz. Bu ağ bizim girdilerimizi (bu örnekte tokenlerimizi) ve eğitilebilir model parametrelerini (ağırlık ve bias parametreleri gibi) alıyor. Model çıktısı ise yine bizim tanımladığımız bir kayıp fonksiyonuna (Loss Function) girerek tek bir sayıya (Loss) indirgeniyor. Eğitim sırasında bizim amacımız bu sayıyı (Loss) düşürmeye çalışmak. Bunu da model parametrelerini ufak ufak artırıp azaltarak yapıyoruz.

Peki hangi parametreleri artırıp hangilerini azaltmamız gerektiğini nereden bileceğiz? İşte bunu **türev** bize bu ilişkiyi veriyor. Bir parametreyi "bir tık" artırdığımızda diğer bir değerinin ne kadar değişeceğinin ölçüsüne türev diyoruz. Gradyan ise türevin çok değişkenli haline verilen isim. Yani modeldeki bütün parametrelerin ne yönde değişmesi gerektiği **Gradyan vektörü** ile tanımlanıyor.

Tüm bu türev hesaplamalarını kendimiz yapabilirdik. Neyse ki Otomatik Türev (AutoGrad) denilen mucizevi yöntem sayesinde bütün bu türev hesapları otomatik yapılabiliyor. Biz ise sadece modeli tanımlayan o hesaplama grafiğini tanımlıyoruz. `Pytorch` gibi kütüphaneler sayesinde bu model tanımlama işini de bildiğimizz python fonksiyonları yazarak yapabiliyoruz. Autograd arka planda bizim yaptığımız bütün matematiksel işlemlerin bir listesini tutuyor ve türevlerini hesaplıyor. Matematiğe hiç elimizi bile sürmeden matematiksel olarak hesaplaması gayet zor olacak şeyleri farkına bile varmadan yapabiliyoruz. İşte Yapay Zekanın son yıllarda bu kadar hızlı ilerlemesinin sebeplerinden biri de bu.

Bu tür kütüphaneleri kullananlar bilecektir ki modelin yapacağı işlemleri `forward` adlı bir fonksiyon tanımlıyoruz. Bu fonksiyon, modelin (hesaplama grafiğinin) **ileri yönde** (forward) yani girdilerden çıktıya doğru hangi işlemleri yapacağını tanımlıyor.

Geri yayılım algoritması ise geri yönde, yani kayıp değerinden başlayarak girdilere doğru geriye giderek her parametreye göre Lossun türevlerini oluşturuyor. Bu türevleri calculustaki **zincir kuralı**nı kullanarak yapıyor. Biz ise kodda `backward` fonksiyonunu çağırarak hesaplatıyoruz. İşte Autograd bu backward fonksiyonunu bizim için otomatik olarak yaratıyor.

Fakat bu örnekte hazır hiçbir şey kullanmadığımız için Autograd'ı da kendimiz yazacağız. Korkmayın, sandığınızdan daha kolay.

### Ev Yapımı Autograd kodunun açıklaması

Öncelikle hesaplama grafiğimizdeki her bir düğümü temsilen `Value` sınıfı oluşturuyoruz. Bu sınıfın kendi içinde 4 darklı değer tutacak. `data` bu düğümün hesaplanan skaler değeri, `grad` lossun bu düğüme göre türevi, `_children` (çocuklar) bu düğümün hesaplanmasında kullanılan (ondan önceki katmandan gelen) düğümler, `_local_grads` ise bu düğümün çocuklarına göre yerel türevleri.

Bu Value nesneleri ile yaptığımız her matematiksel işlemde yeni bir Value yaratıyoruz. Örneğin `c = a + b` işlemini tanımladığımızda c Value'sunu yaratıyoruz. c'nin çocukları a ve b Value objeleri. a ve b'nin .data değişkeninin 3 ve 4 oldupunu düşünelim. c.data bu durumda 7 olarak hesaplanıyor. Bu hesaplamayı yaptığımız + işlemi Value sınıfındaki __add__ fonksiyonunda ile tanımlanıyor. Biz bu __add__ fonksiyonunu aşağıda kendimiz tanımlıyoruz. Fakat tanımlarken sadece toplama işlemi yapmakla kalmıyoruz. Yeni bir c Value'su yaratıp onu döndürüyoruz. c.data elbette a + b toplamı yani 7 oluyor. c'nin çocukları olarak a ve b'yi kaydediyoruz. Ama en önemlisi toplama işlemi için için c'nin a'ya ve b'ye göre 'yerel' türevlerini tanımlayıp onları da c'ye keydediyoruz. Toplama için her iki türev de 1 (a + b'nin a'ya ve b'ye göre türevleri 1 ve 1).

Bunu Value'ları kullanarak için yapmak istediğimiz bütün temel matematiksel işlemler için ayrı ayrı tanımlamamız gerekiyor. Bunların birleşiminden oluşan daha karmaşık işlemleri tekrar tanımlamamıza gerek yok. Buradaki GPT örneği için tablodaki tanımlar yetiyor:

| İşlem | İleri | Yerel Gradyanlar |
|-----------|----------|-------------------|
| `a + b`   | $$a + b$$ | $$\frac{\partial}{\partial a}=1,\quad \frac{\partial}{\partial b}=1$$ |
| `a * b`   | $$a \cdot b$$ | $$\frac{\partial}{\partial a}=b,\quad \frac{\partial}{\partial b}=a$$ |
| `a ** n`  | $$a^n$$ | $$\frac{\partial}{\partial a}=n a^{n-1}$$ |
| `log(a)`  | $$\ln(a)$$ | $$\frac{\partial}{\partial a}= \frac{1}{a}$$ |
| `exp(a)`  | $$e^a$$ | $$\frac{\partial}{\partial a}= e^a$$ |
| `relu(a)` | $$\max(0,a)$$ | $$\frac{\partial}{\partial a}= \mathbf{1}_{a>0}$$ |

#### `.backward` fonksiyonu:
Dikkat ederseniz Value nesnelerini yaratırken (__init__) .grad değerlerini 0 olarak bıraktık. Bu degerleri yani gradyanları hesaplamak için backward fonksiyonunu kullanıyoruz. Örneğin loss.backward() şeklinde çağırdığımızda en sondaki loss Value'sundan başlayarak, sondan başa doğru, hesaplama grafiğindeki bütün Value düğümlerini geziyoruz ve grad değerlerini zincir kuralını kullanarak hesaplıyoruz.

backward fonksiyonu içinde ilk yaptığımız iş bu tersine gezme sırasını oluşturmak. Buna tür bir sıralamaya tersine topolojik sıralama deniyor. Burada DFS (Depth First Search, Derinlik Öncelikli Arama) adlı klasik algoritmanın bir versiyonu kullanılarak topolojik sıralama bulunuyor (topo). DFS sırasında eğer bir düğümün bütün çocukları gezildiyse toplo listesine ekliyoruz.

Loss'tan başlıyoruz ve onun kendine göre türevi 1. Yani en sonraki düğümün grad değeri 1 oluyor.

Sonrasında `v in reversed(topo)` ile sondan başa doğru v düğümleri geziliyor. Burada yapılan işlem önemli.

v'nin .grad değeri yerel gradyanlar ile çarpılarak çocuklara aktarılıyor.  Burada çocukların mevcut gradyanına v'den yeni gelen gradyanlar ekleniyor. Bunun sebebi ağda dallanmalar olabilmesi ve bir düğüme birden fazla koldan gradyan akabilmesi.

backward tamamlandığında her düğümün grad değeri hesaplanmış oluyor. Bu grad degeri $\frac{\partial L}{\partial v}$'yı, yani v.data'yı bir tık degistirirsek kayip degerinin ne yönde kaç tık değişeceğini gösteriyor.

---
Daha fazla ayrıntı istiyorsanız Karphathy'nin 2 buçuk saatlik [micrograd](https://www.youtube.com/watch?v=VMj-3S1tku0) videosunu izleyebilirsiniz.


In [ ]:

# Hesaplama grafiği üzerinden zincir kuralını özyinelemeli uygulayan Autograd olsun
class Value:
    __slots__ = ('data', 'grad', '_children', '_local_grads') # bellek kullanımı için Python optimizasyonu

    def __init__(self, data, children=(), local_grads=()):
        self.data = data                # ileri geçişte (forward) hesaplanan bu düğümün skaler değeri
        self.grad = 0                   # geri geçişte (backward) kaybın bu düğüme göre türevi
        self._children = children       # hesaplama grafiğinde bu düğümün çocukları
        self._local_grads = local_grads # bu düğümün çocuklarına göre yerel türevleri

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return Value(self.data + other.data, (self, other), (1, 1))

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return Value(self.data * other.data, (self, other), (other.data, self.data))

    def __pow__(self, other): return Value(self.data**other, (self,), (other * self.data**(other-1),))
    def log(self): return Value(math.log(self.data), (self,), (1/self.data,))
    def exp(self): return Value(math.exp(self.data), (self,), (math.exp(self.data),))
    def relu(self): return Value(max(0, self.data), (self,), (float(self.data > 0),))
    def __neg__(self): return self * -1
    def __radd__(self, other): return self + other
    def __sub__(self, other): return self + (-other)
    def __rsub__(self, other): return other + (-self)
    def __rmul__(self, other): return self * other
    def __truediv__(self, other): return self * other**-1
    def __rtruediv__(self, other): return other * self**-1

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._children:
                    build_topo(child)
                topo.append(v) # not: bütün çocuklarını tamamladıktan sonra listeye ekleyebilirsin
        build_topo(self)
        self.grad = 1
        for v in reversed(topo):
            for child, local_grad in zip(v._children, v._local_grads):
                child.grad += local_grad * v.grad


In [ ]:

# Model bilgisini tutacak parametreleri başlat.
n_embd = 16     # embedding boyutu
n_head = 4      # attention head sayısı
n_layer = 1     # katman sayısı
block_size = 16 # maksimum dizi uzunluğu
head_dim = n_embd // n_head # her head’in boyutu
matrix = lambda nout, nin, std=0.08: [[Value(random.gauss(0, std)) for _ in range(nin)] for _ in range(nout)]
state_dict = {'wte': matrix(vocab_size, n_embd), 'wpe': matrix(block_size, n_embd), 'lm_head': matrix(vocab_size, n_embd)}
for i in range(n_layer):
    state_dict[f'layer{i}.attn_wq'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.attn_wk'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.attn_wv'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.attn_wo'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.mlp_fc1'] = matrix(4 * n_embd, n_embd)
    state_dict[f'layer{i}.mlp_fc2'] = matrix(n_embd, 4 * n_embd)
params = [p for mat in state_dict.values() for row in mat for p in row] # parametreleri tek bir liste[Value] halinde düzleştir
print(f"parametre sayısı: {len(params)}")

# Model mimarisini tanımla: token dizisi + parametreler -> bir sonraki token için logits.
# GPT-2’yi takip et, (GPT’ler arasında kutsanmış), küçük farklarla: layernorm -> rmsnorm, bias yok, GeLU -> ReLU
def linear(x, w):
    return [sum(wi * xi for wi, xi in zip(wo, x)) for wo in w]

def softmax(logits):
    max_val = max(val.data for val in logits)
    exps = [(val - max_val).exp() for val in logits]
    total = sum(exps)
    return [e / total for e in exps]

def rmsnorm(x):
    ms = sum(xi * xi for xi in x) / len(x)
    scale = (ms + 1e-5) ** -0.5
    return [xi * scale for xi in x]

def gpt(token_id, pos_id, keys, values):
    tok_emb = state_dict['wte'][token_id] # token embedding’i
    pos_emb = state_dict['wpe'][pos_id] # pozisyon embedding’i
    x = [t + p for t, p in zip(tok_emb, pos_emb)] # token + pozisyon embedding’i
    x = rmsnorm(x)

    for li in range(n_layer):
        # 1) Multi-head attention bloğu
        x_residual = x
        x = rmsnorm(x)
        q = linear(x, state_dict[f'layer{li}.attn_wq'])
        k = linear(x, state_dict[f'layer{li}.attn_wk'])
        v = linear(x, state_dict[f'layer{li}.attn_wv'])
        keys[li].append(k)
        values[li].append(v)
        x_attn = []
        for h in range(n_head):
            hs = h * head_dim
            q_h = q[hs:hs+head_dim]
            k_h = [ki[hs:hs+head_dim] for ki in keys[li]]
            v_h = [vi[hs:hs+head_dim] for vi in values[li]]
            attn_logits = [sum(q_h[j] * k_h[t][j] for j in range(head_dim)) / head_dim**0.5 for t in range(len(k_h))]
            attn_weights = softmax(attn_logits)
            head_out = [sum(attn_weights[t] * v_h[t][j] for t in range(len(v_h))) for j in range(head_dim)]
            x_attn.extend(head_out)
        x = linear(x_attn, state_dict[f'layer{li}.attn_wo'])
        x = [a + b for a, b in zip(x, x_residual)]
        # 2) MLP bloğu
        x_residual = x
        x = rmsnorm(x)
        x = linear(x, state_dict[f'layer{li}.mlp_fc1'])
        x = [xi.relu() for xi in x]
        x = linear(x, state_dict[f'layer{li}.mlp_fc2'])
        x = [a + b for a, b in zip(x, x_residual)]

    logits = linear(x, state_dict['lm_head'])
    return logits

# Adam olsun, kutsanmış optimize edici ve tamponları
learning_rate, beta1, beta2, eps_adam = 0.01, 0.85, 0.99, 1e-8
m = [0.0] * len(params) # birinci moment tamponu
v = [0.0] * len(params) # ikinci moment tamponu

# Sırayla tekrarla
num_steps = 1000 # eğitim adımı sayısı
for step in range(num_steps):

    # Tek bir doküman al, tokenize et, iki yanına BAS özel token’ını koy
    doc = docs[step % len(docs)]
    tokens = [BAS] + [uchars.index(ch) for ch in doc] + [BAS]
    n = min(block_size, len(tokens) - 1)

    # Token dizisini modelden geçir, kayba kadar tüm hesaplama grafiğini kur.
    keys, values = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
    losses = []
    for pos_id in range(n):
        token_id, target_id = tokens[pos_id], tokens[pos_id + 1]
        logits = gpt(token_id, pos_id, keys, values)
        probs = softmax(logits)
        loss_t = -probs[target_id].log()
        losses.append(loss_t)
    loss = (1 / n) * sum(losses) # doküman dizisi üzerindeki nihai ortalama kayıp. Düşük olsun.

    # Kaybı geri yay, tüm model parametrelerine göre gradyanları hesapla.
    loss.backward()

    # Adam güncellemesi: gradyanlara göre model parametrelerini güncelle.
    lr_t = learning_rate * (1 - step / num_steps) # doğrusal öğrenme oranı azaltımı
    for i, p in enumerate(params):
        m[i] = beta1 * m[i] + (1 - beta1) * p.grad
        v[i] = beta2 * v[i] + (1 - beta2) * p.grad ** 2
        m_hat = m[i] / (1 - beta1 ** (step + 1))
        v_hat = v[i] / (1 - beta2 ** (step + 1))
        p.data -= lr_t * m_hat / (v_hat ** 0.5 + eps_adam)
        p.grad = 0

    print(f"adım {step+1:4d} / {num_steps:4d} | kayıp {loss.data:.4f}")

# Çıkarım (Inference): model bize geri “gevezelesin”
temperature = 0.5 # (0, 1] aralığında, üretilen metnin “yaratıcılığını” kontrol eder, düşükten yükseğe
print("\n--- çıkarım (yeni, halüsinasyon isimler) ---")
for sample_idx in range(20):
    keys, values = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
    token_id = BAS
    sample = []
    for pos_id in range(block_size):
        logits = gpt(token_id, pos_id, keys, values)
        probs = softmax([l / temperature for l in logits])
        token_id = random.choices(range(vocab_size), weights=[p.data for p in probs])[0]
        if token_id == BAS:
            break
        sample.append(uchars[token_id])
    print(f"örnek {sample_idx+1:2d}: {''.join(sample)}")